In [ ]:
import os, sys
from larcv import larcv
larcv.load_pyutil
larcv.load_cvutil
import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
pygeo = geo2d.PyDraw()

from ROOT import larocv
from ROOT import std
from ROOT import cv
from ROOT.cv import Point_ as Vector
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
%matplotlib inline
import numpy as np
import math

colors = ["blue", "green", "red", "cyan", "magenta", "yellow"]
vertex_type = ["kUnknown", "kTime", "kWire",   "kShower", "kEndOfTrack", "kEdge"]

In [ ]:
proc = larcv.ProcessDriver('ProcessDriver')
CFG="../cfg/prod_fullchain_ssnet_combined_test.cfg"

preprocessed=False

print "Loading config... ",CFG
proc.configure(CFG)
flist=ROOT.std.vector('std::string')()
flist.push_back("/Users/dayajun/Desktop/input/intrinsic_nue/fcn_all.root")
proc.override_input_file(flist)
proc.override_output_file("/tmp/cacca.root")
proc.override_ana_file("/tmp/test.root")

vinroi_id  = proc.process_id("VertexInROI")
reco_id    = proc.process_id("LArbysImage")
larbysimg  = proc.process_ptr(reco_id)

proc.initialize()

In [ ]:
event=0#0,5(fixed), 8(fixed),11,12,13,21(too hard),14(fxxk),16,17(have something to do with start radius)
proc.batch_process(event,1)


In [ ]:
mgr=larbysimg.Manager()  
dm=mgr.DataManager()
mid = 0
for module in dm.Names():
    print mid,",", module
    mid+=1

In [ ]:
img_v  = [None,None,None]
oimg_v = [None,None,None]
track_img_v  = [None,None,None]
otrack_img_v = [None,None,None]
shower_img_v  = [None,None,None]
oshower_img_v = [None,None,None]

for ix in xrange(3):
    oimg_v[ix]        = pygeo.image(mgr.OriginalInputImages(0)[ix])
    otrack_img_v[ix]  = pygeo.image(mgr.OriginalInputImages(1)[ix])
    oshower_img_v[ix] = pygeo.image(mgr.OriginalInputImages(2)[ix])
    img_v[ix]         = pygeo.image(mgr.InputImages(0)[ix])
    track_img_v[ix]   = pygeo.image(mgr.InputImages(1)[ix])
    shower_img_v[ix]  = pygeo.image(mgr.InputImages(2)[ix])

# ShapeAnalysis

In [ ]:
assman=dm.AssManager()
algoid=dm.ID("combinevertex")
data=dm.Data(algoid,0)
vtx_data=dm.Data(algoid,0).as_vector()

print 'has # of vertices', vtx_data.size()
for vtxid,vtx in enumerate(vtx_data):
    print "vertex type:",vertex_type[vtx.type]
    print "<===================Start dQds on vertex_id ",vtxid," ==========================>"
    for plane in xrange(3):
        print "<===================plane ", plane,"================================>"
        
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        nz_pixels=np.where(shape_img>0.0)

        algoid=dm.ID("angleanalysis")
        par_data=dm.Data(algoid,plane)
        
        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))# Get assciated particle
        
        if ass_t.size!=2:
            print "plane ",plane, "has",ass_t.size,"particles coming out"
            continue
        
        fig,ax = plt.subplots(figsize=(8,8),facecolor='w')
        ax.imshow(shape_img,cmap='Greys',interpolation='none')

        par_data_v=par_data.as_vector()
        x_2d = vtx.vtx2d_v[plane].pt.x
        y_2d = vtx.vtx2d_v[plane].pt.y    
        
        cid = 0# color_idx
        vtx_dqds = 0
        
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            pca  = larocv.CalcPCA(par_data_v[id_]._ctor)
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=4,alpha=0.8, color = colors[cid])
            
            cir = par_data_v[id_]._circle
            cid += 1
                    
        ax.plot(x_2d, y_2d,'*',color=colors[-1],markersize=20)
        ax.set_title("VertexID:%i, Vertex Type: %d on plane %i"%(vtxid,vtx.type, plane))

        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        plt.show()
        print "<=============================================================================>"
    print "<===================End combined vertex ",vtxid," ==========================>"